In [ ]:
import json

file1 = "sparse_probing/src/sparse_probing_results/sae_bench_pythia70m_sweep_topk_ctx128_0730_eval_results.json"
file2 = "sparse_probing/src/sparse_probing_results/3_sae_bench_pythia70m_sweep_topk_ctx128_0730_eval_results.json"

with open(file1, "r") as f:
    data1 = json.load(f)

with open(file2, "r") as f:
    data2 = json.load(f)



In [ ]:

def compare_dicts_within_tolerance(actual, expected, tolerance, path="", all_diffs=None):
    """
    Recursively compare two nested dictionaries and assert that all numeric values
    are within the specified tolerance. Print global mean and max difference at root call.

    :param actual: The actual dictionary of results
    :param expected: The expected dictionary of results
    :param tolerance: The allowed tolerance for floating point comparisons
    :param path: The current path in the nested structure (used for error messages)
    :param all_diffs: List to collect all differences (used internally for recursion)
    """
    if all_diffs is None:
        all_diffs = []

    assert type(actual) == type(
        expected
    ), f"Type mismatch at {path}: {type(actual)} != {type(expected)}"

    if isinstance(actual, dict):
        assert set(actual.keys()) == set(
            expected.keys()
        ), f"Key mismatch at {path}: {set(actual.keys())} != {set(expected.keys())}"
        for key in actual:
            new_path = f"{path}.{key}" if path else str(key)

            if key == "acc_0" or key == "acc_1" or key == "loss":
                continue

            compare_dicts_within_tolerance(
                actual[key], expected[key], tolerance, new_path, all_diffs
            )
    elif isinstance(actual, (int, float)):
        diff = abs(actual - expected)
        all_diffs.append(diff)
    else:
        assert actual == expected, f"Value mismatch at {path}: {actual} != {expected}"

    # Print global mean and max difference only at the root call
    if path == "":
        if all_diffs:
            mean_diff = sum(all_diffs) / len(all_diffs)
            max_diff = max(all_diffs)

            print(f"Global mean difference: {mean_diff}")
            print(f"Global max difference: {max_diff}")

            assert (
                max_diff <= tolerance
            ), f"Value mismatch at {path}: {actual} not within {tolerance} of {expected}"

        else:
            print("No numeric differences found.")

compare_dicts_within_tolerance(data1, data2, 1e-5)